# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397732184568                   -0.90    5.2   25.9ms
  2   -8.400233566072       -2.60       -1.74    1.0   18.4ms
  3   -8.400398587249       -3.78       -2.92    1.8   50.2ms
  4   -8.400427822393       -4.53       -2.93    3.0   24.0ms
  5   -8.400427868944       -7.33       -2.93    1.0   18.6ms
  6   -8.400428146827       -6.56       -4.59    1.0   18.5ms
  7   -8.400428155666       -8.05       -4.35    2.8   23.3ms
  8   -8.400428156202       -9.27       -4.79    1.0   35.0ms
  9   -8.400428156274      -10.14       -6.28    1.0   18.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397838971365                   -0.90           5.0    453ms
  2   -8.400385434266       -2.59       -1.79   0.80    2.0    532ms
  3   -8.400422984695       -4.43       -3.02   0.80    1.0    185ms
  4   -8.400428119230       -5.29       -3.41   0.80    2.5   21.0ms
  5   -8.400428153645       -7.46       -4.52   0.80    1.0   16.5ms
  6   -8.400428156251       -8.58       -5.78   0.80    2.5   20.1ms
  7   -8.400428156277      -10.59       -6.29   0.80    2.8   21.8ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.762041365267                   -1.07    2.73s
  2   -1.579307828432        0.37       -0.64    107ms
  3   -3.716940096046        0.33       -0.40   39.0ms
  4   -4.598620338247       -0.05       -0.47   68.3ms
  5   -6.578335784565        0.30       -0.55   39.0ms
  6   -7.438588738083       -0.07       -0.85   43.9ms
  7   -7.464910342063       -1.58       -1.37   29.0ms
  8   -7.903745996892       -0.36       -1.58   49.0ms
  9   -8.246740054366       -0.46       -1.22   39.2ms
 10   -8.322512551839       -1.12       -1.71   29.1ms
 11   -8.365841057723       -1.36       -1.95   29.1ms
 12   -8.378132747129       -1.91       -2.55   28.8ms
 13   -8.391249979460       -1.88       -2.44   32.7ms
 14   -8.395183572896       -2.41       -2.75   28.8ms
 15   -8.398206005343       -2.52       -2.89   28.9ms
 16   -8.398568622338       -3.44       -3.34   32.1ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397830382981                   -0.90    5.0   40.8ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427993866                   -1.79    23.7s
  2   -8.400428156277       -6.79       -4.04    2.10s
  3   -8.400428156277      -14.45       -7.85    110ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.2168482113942163e-6
|ρ_newton - ρ_scfv| = 4.4999930180595514e-8
|ρ_newton - ρ_dm|   = 9.842086958354317e-7
